<a href="https://colab.research.google.com/github/SupradeepDanturti/ConvAIProject/blob/dev2/ConvAI_Project_submission_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Project 7: Speaker Counter and Overlap Detector](#scrollTo=TZzv25vDL9LQ)

>>[Abstract](#scrollTo=MP7fynuIMGlg)

>>[Introduction](#scrollTo=wBGDBctaMYYQ)

>>[Methodology](#scrollTo=XtzOOJDnMpWp)

>>>>[Preparation & Preprocessing](#scrollTo=XtzOOJDnMpWp)

>>>>[Data Augmentation](#scrollTo=XtzOOJDnMpWp)

>>>>[Model Development and Optimization](#scrollTo=XtzOOJDnMpWp)

>>>>[ECAPA TDNN](#scrollTo=XtzOOJDnMpWp)

>>>>[XVector](#scrollTo=XtzOOJDnMpWp)

>>>>[Selfsupervised - MLP](#scrollTo=XtzOOJDnMpWp)

>>>>[Selfsupervised XVector](#scrollTo=XtzOOJDnMpWp)

>>>>[Original Aspects and Relation to Existing Work](#scrollTo=XtzOOJDnMpWp)

>>[Experimental Setup](#scrollTo=YIQOcLeaPq3v)

>>>[Model Details](#scrollTo=YIQOcLeaPq3v)

>>[Model Performance Analysis - Results](#scrollTo=BRL5KR20QWKu)

>>>>[X-Vector Model](#scrollTo=BRL5KR20QWKu)

>>>>[ECAPA-TDNN Model](#scrollTo=BRL5KR20QWKu)

>>>>[Self-Supervised MLP Model](#scrollTo=BRL5KR20QWKu)

>>>>[Self-Supervised X-Vector Model](#scrollTo=BRL5KR20QWKu)

>>>>[Classwise Error Rate](#scrollTo=BRL5KR20QWKu)

>>[Steps Download, Create and Train all models](#scrollTo=OTaD2SZ2cvsR)

>>[Inference Interface](#scrollTo=vpsZKEbeko19)

>>[Conclusions](#scrollTo=b4Jyn3BcQDpf)

>>[References](#scrollTo=yaxqlm6kRcmb)



# **Project 7: Speaker Counter and Overlap Detector**

## **Abstract**

 - In this project, we address the challenge of accurately counting speakers in meeting recordings, a task complicated by frequent instances of overlapping speech. Leveraging speech separation technologies such as SepFormer, implemented in SpeechBrain, the objective was to initially identify segments containing overlapping speech before applying speech separation. To achieve this, we undertook a comprehensive review of literature on speaker counting and developed a data simulator. This simulator generated overlapping speech signals by combining clean data from the LibriSpeech-Clean-100 dataset with noise and reverberation from the Open-RIR dataset, based on a specified probability.

 - We experimented with and evaluated two primary models for the task of speaker counting: X-Vector and ECAPA-TDNN, with the input being 1-2 second speech segments. Additionally, a pretrained Wav2Vec 2.0 model coupled with a multilayer perceptron (MLP) classifier was explored. The models were tasked with processing long recordings, segmented into 1-2 second chunks, and determining the number of speakers in each segment. The inference results were outputted as a text file, detailing the start and end times of each segment alongside the speaker count.

-  The ECAPA-TDNN model demonstrated the lowest error rate among the tested models, showcasing its effectiveness in distinguishing between zero to three or more speakers within the overlapping speech segments. This research not only contributes a practical tool for speaker counting in complex audio environments but also integrates this functionality into the SpeechBrain project, enhancing its utility for speech technology applications.


## **Introduction**

 -In the era of digital communication, the ability to accurately process and interpret spoken language in audio recordings is becoming increasingly vital. One significant challenge in this field is the problem of speaker counting and diarization in environments where multiple individuals speak simultaneously. This issue is particularly prevalent in meeting scenarios, where overlapping speech can significantly complicate transcription and analysis tasks, leading to errors in speaker identification and speech recognition.

- The importance of resolving this challenge cannot be overstated, as it directly impacts the effectiveness of automated systems in various applications, from virtual assistant technologies to legal and medical transcription services. The main challenges in tackling overlapping speech include the segmentation of speech, identification of individual speakers, and the subsequent processing of these segments to accurately count and distinguish between speakers.

- Prior works have explored various aspects of speaker diarization and counting. Notably, **Bredin et al.** (2020) introduced *pyannote.audio*, a toolkit that provides neural building blocks specifically designed for speaker diarization tasks. This framework has facilitated significant advancements in the field by enabling researchers and developers to implement and experiment with complex neural network architectures for audio processing tasks.

- Approach builds upon these foundational studies by employing advanced neural network models to address the complexities of speaker counting within overlapping speech segments. We focus particularly on the application of X-Vector and ECAPA-TDNN architectures, renowned for their efficacy in speaker verification tasks, and explore the integration of a pretrained Wav2Vec 2.0 model coupled with a multilayer perceptron (MLP) for classification.

- To train and evaluate the models, we utilized a simulated dataset created by mixing clean speech samples from the LibriSpeech-Clean-100 dataset with noise and reverberation from the Open-RIR dataset. This setup allowed us to mimic real-world overlapping speech scenarios and provided a robust platform for testing the models' effectiveness.

- Preliminary results have been promising, particularly with the use of pretrained(Wav2Vec2) model, which demonstrated the lowest error rate among the models tested. This outcome suggests that the approach could significantly enhance the accuracy and reliability of speaker counting systems in real-world applications.

- The adoption of these advanced methodologies not only pushes the boundaries of what is achievable with current speech processing technology but also sets a new standard for future research and development in the field.

## **Methodology**

My project methodology comprised several significant steps that enabled me to effectively address the complexities of speaker counting within overlapping speech segments. Here, we focus on the crucial aspects of the approach that contributed significantly to the project's success.

#### **Preparation & Preprocessing**
- Used the [LibriSpeech-Clean-100](https://www.openslr.org/12) dataset for clean speech samples and [Open-RIR](https://www.openslr.org/28/) dataset for realistic noise and reverberation, creating a challenging environment for the models. For Evaluation used [LibriSpeech-Dev-Clean](http://www.openslr.org/resources/12/dev-clean.tar.gz) and tested on [LibriSpeech-test-Clean](http://www.openslr.org/resources/12/test-clean.tar.gz).  Each audio file was segmented into 1-2 second clips, which were then annotated with the number of speakers present, ranging from 0 to 4.

#### **Data Augmentation**
- Data Augmentation played a critical role in enhancing the robustness of the models. Implemented techniques such as noise injection, varying speed, and pitch modification to ensure that the models could generalize well across different acoustic environments and speaker variations.

#### **Model Development and Optimization**
- I have developed and tested four models: X-Vector, ECAPA-TDNN, and a combination of Pretrained Wav2Vec 2.0 with MLP and with Xvectors. Each model was chosen based on its proven efficacy in related tasks such as speaker verification and identification.

#### 1. ECAPA TDNN

<center>
<img src="https://drive.google.com/uc?export=view&id=1IZHqF86vornEw9Ib-N46JMVyhMZ7cctZ" alt="Image description" width="500"/>
<figcaption>Fig.1 - ECAPA-TDNN</figcaption>
</center>

#### 2. XVector
<center>
<img src="https://drive.google.com/uc?export=view&id=1n6mRBqFzJNfzbmzQjcKQmDjZBTxr0hbN" alt="Image description" width="500"/>
<figcaption>Fig.2 - XVector</figcaption>
</center>

#### 3. Selfsupervised - MLP
<center>
<img src="https://drive.google.com/uc?export=view&id=1enhfLUxl3v-FWZ3bLNa0tFmGLvF6VAvq" alt="Image description" width="500"/>
<figcaption>Fig.3 - wav2vec2 with linear classifier</figcaption>
</center>

#### 4. Selfsupervised XVector
<center>
<img src="https://drive.google.com/uc?export=view&id=1en7t4gevL-3QqA5ru6Mz7VlfpGx7cby3" alt="Image description" width="500"/>
<figcaption>Fig.4 - wav2vec2 with Xvector</figcaption>
</center>

#### **Original Aspects and Relation to Existing Work**
- My approach extends existing methodologies by integrating robust data augmentation and advanced neural architectures specifically tuned for the task of speaker counting in overlapping speech conditions. This project not only adapts existing frameworks like those introduced in [pyannote.audio](https://huggingface.co/pyannote/overlapped-speech-detection) but also pushes forward the state-of-the-art by focusing on quantitative metrics such as error rates in realistic, noisy environments.

## **Experimental Setup**
### Model Details

<center>
<table>
  <tr>
    <th>Model</th>
    <th>Hyperparams</th>
    <th>GitHub Link</th>
    <th>Model</th>
    <th>Hyperparams</th>
    <th>GitHub Link</th>
  </tr>
  <tr>
    <td>ECAPA-TDNN</td>
    <td><pre><code>
    sample_rate: 16000
    number_of_epochs: 20
    batch_size: 64
    lr_start: 0.001
    lr_final: 0.0001
    weight_decay: 0.00002
    num_workers: 0 # For windows or 4 for linux
    n_classes: 5
    dim: 192
    num_attention_channels: 128
    n_mels: 80
    channels: [256, 256, 256, 256, 768]
    kernel_sizes: [5, 3, 3, 3, 1]
    dilations: [1, 2, 3, 4, 1]
    </code></pre></td>
    <td>[view complete file]()</td>
    <td>SelfSupervised <br>
    Linear Classifier</td>
    <td><pre><code>
    number_of_epochs: 5
    batch_size: 64
    lr: 0.001
    lr_ssl: 0.0001
    freeze_ssl: False
    freeze_ssl_conv: True
    encoder_dim: 768
    out_n_neurons: 5
    </code></pre></td>
    <td>[view complete file]()</td>
  </tr>
  <tr>
    <td>XVector</td>
    <td><pre><code>
    sample_rate: 16000
    number_of_epochs: 50
    batch_size: 64
    lr_start: 0.001
    lr_final: 0.0001
    weight_decay: 0.00002
    num_workers: 0 # For windows or 4 for linux
    n_mels: 4
    n_classes: 5
    emb_dim: 128
    tdnn_channels: 64
    tdnn_channels_out: 128
    tdnn_kernel_sizes: [5, 3, 3, 1, 1]
    tdnn_dilations: [1, 2, 3, 1, 1]
    </code></pre></td>
    <td>[view complete file]()</td>
    <td>SelfSupervised <br>
    XVector</td>
    <td><pre><code>
    number_of_epochs: 15
    batch_size: 128
    lr: 0.001
    lr_final: 0.0001
    lr_ssl: 0.00001
    freeze_ssl: False
    freeze_ssl_conv: True
    encoder_dim: 768
    emb_dim: 128
    out_n_neurons: 5
    tdnn_channels: [ 64, 64, 64 ]
    tdnn_kernel_sizes: [ 5, 2, 3 ]
    tdnn_dilations: [ 1, 2, 3 ]
    </code></pre></td>
    <td>[view complete file]()</td>
  </tr>
</table></center>

## **Model Performance Analysis - Results**

#### **X-Vector Model**
The X-Vector model's training showcased an initial decline in loss, with the augmented data presenting a consistent downward trend indicative of effective learning. However, this was coupled with fluctuations that raise concerns of possible overfitting to the augmented noises. The unaugmented dataset displayed a higher error rate that remained stable across epochs, suggesting a potential limitation in the model's ability to generalize to the augmented conditions without additional optimization strategies.
<center>
<table>
  <tr>
    <td>
      <img src="https://drive.google.com/uc?export=view&id=1t1VZhDyG8awIU0keoyrw54yRLETa7r9B" alt="Selfsupervised XVector Train and Valid Loss" width="300"/>
      <figcaption>Fig.5 - XVector Train and Valid Loss</figcaption>
    </td>
    <td>
      <img src="https://drive.google.com/uc?export=view&id=1qVfJx5yzxa_UKE3XVAveicqboy69l2kV" alt="img" width="300"/>
      <figcaption>Fig.6 - Error rate of Both XVector Models</figcaption>
    </td>
  </tr>
</table>
</center>

#### **ECAPA-TDNN Model**
The ECAPA-TDNN model demonstrated a downward trend in error rates for both the augmented and unaugmented datasets, with the augmented setup yielding notably lower error rates. This underscores the model's capacity to generalize from augmented training. However, fluctuations in the augmented validation loss suggest sensitivity to novel complex patterns, indicating potential overfitting and necessitating further model refinement.

<center>
<table>
  <tr>
    <td>
      <img src="https://drive.google.com/uc?export=view&id=1VeKXeO3-aXcTyBz6Ztq-WH3M40njXEkq" alt="Selfsupervised XVector Train and Valid Loss" width="300"/>
      <figcaption>Fig.7 - ECAPA-TDNN Train and Valid Loss</figcaption>
    </td>
    <td>
      <img src="https://drive.google.com/uc?export=view&id=1TQ3qK56wQr3lxTEh9-NUh5s-NtRkqtyB" alt="img" width="300"/>
      <figcaption>Fig.8 - Error rate of Both ECAPA-TDNN Models</figcaption>
    </td>
  </tr>
</table>
</center>

#### **Self-Supervised MLP Model**
The self-supervised MLP model, powered by Wav2Vec 2.0, exhibited a promising decline in training loss, though the reduction in validation loss plateaued early, suggesting a limit to its learning capacity in this context. While it did not reach the lowest error rates among the models, it demonstrates the effectiveness of self-supervised learning frameworks in achieving rapid initial improvements.

<center>
<img src="https://drive.google.com/uc?export=view&id=1IgCQqGb1HLbyrhYwUMPuV8fjY-Xy4aUy" alt="Selfsupervised MLP Train and Valid Loss" width="300"/>
<figcaption>Fig.9 - Self-Supervised MLP Train and Valid Loss</figcaption>
</center>

#### **Self-Supervised X-Vector Model**
The self-supervised X-Vector model exhibited outstanding performance with the lowest error rates, validating the robust feature extraction capability of the Wav2Vec 2.0 framework. Despite some fluctuations in the error rate, the overall trend displayed an impressive adaptability to speaker nuances, highlighting the advantages of pretraining on a vast corpus of unlabeled data.

<center>
<table>
  <tr>
    <td>
      <img src="https://drive.google.com/uc?export=view&id=1K6vvfvfoIZShz8Y1lV-epE2waoPgdMIs" alt="Selfsupervised XVector Train and Valid Loss" width="300"/>
      <figcaption>Fig.10 - Self-Supervised X-Vector Train and Valid Loss</figcaption>
    </td>
    <td>
      <img src="https://drive.google.com/uc?export=view&id=1cZDKGp0VBPOE9TKhWkUXCnd7bgrXV4IQ" alt="img" width="300"/>
      <figcaption>Fig.11 - Error rate of Both Selfsupervised Models</figcaption>
    </td>
  </tr>
</table>
</center>
<br>

#### **Classwise Error Rate**
<center>
<table>
 <tr>
    <th>Model</th>
    <th>Class</th>
    <th>Error Rate</th>
    <th>Model</th>
    <th>Class</th>
    <th>Error Rate</th>
  </tr>
  <tr>
    <td rowspan="5">XVector</td>
    <td>No Speakers</td>
    <td>1e-05</td>
    <td rowspan="5">ECAPA-TDNN</td>
    <td>No Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>1 Speaker</td>
    <td>1e-05</td>
    <td>1 Speaker</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>2 Speakers</td>
    <td>1e-05</td>
    <td>2 Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>3 Speakers</td>
    <td>1e-05</td>
    <td>3 Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>4 Speakers</td>
    <td>1e-05</td>
    <td>4 Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td rowspan="5">Selfsupervised MLP</td>
    <td>No Speakers</td>
    <td>1e-05</td>
    <td rowspan="5">Selfsupervised XVector</td>
    <td>No Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>1 Speaker</td>
    <td>1e-05</td>
    <td>1 Speaker</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>2 Speakers</td>
    <td>1e-05</td>
    <td>2 Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>3 Speakers</td>
    <td>1e-05</td>
    <td>3 Speakers</td>
    <td>1e-05</td>
  </tr>
  <tr>
    <td>4 Speakers</td>
    <td>1e-05</td>
    <td>4 Speakers</td>
    <td>1e-05</td>
  </tr>
</table>
</center>

## **Steps Download, Create and Train all models**

Pull codebase from github
```
!git clone --filter=blob:none --no-checkout https://github.com/SupradeepDanturti/ConvAIProject
%cd ConvAIProject
```

Prepare the dataset

1.   Download Dataset
```
!python prepare_dataset/download_required_data.py --output_folder <destination_folder_path>
```
2.   Create Damples
```
!python prepare_dataset/create_custom_dataset.py dataset.yaml
```
dataset.yaml
```
n_sessions:
  train: 1000 # Creates 1000 sessions per class
  dev: 200 # Creates 200 sessions per class
  eval: 200 # Creates 200 sessions per class
n_speakers: 4 # max number of speakers. In this case the total classes will be 5 (0-4 speakers)
max_length: 120 # max length in seconds for each session/utterance.
```
3. Train XVector Model
```
!python xvector/train_xvector_augmentation.py xvector/hparams_xvector_augmentation.yaml
```
4. Train ECAPA-TDNN Model
```
!python ecapa_tdnn/train_ecapa_tdnn.py ecapa_tdnn/hparams_ecapa_tdnn_augmentation.yaml
```
5. Train Selfsupervised MLP(Linear Classifier)
```
!python selfsupervised/train_selfsupervised_mlp.py selfsupervised/hparams_selfsupervised_mlp.yaml
```
6. Train Selfsupervised XVector
```
!python selfsupervised/train_selfsupervised.py selfsupervised/hparams_selfsupervised_xvector.yaml
```








## **Inference Interface**



```
from SpeakerCounter import SpeakerCounter
wav_path = wav_path
save_dir = save_dir_path
model_path = model_path

audio_classifier = SpeakerCounter.from_hparams(source=model_path, savedir=save_dir)

audio_classifier.classify_file(wav_path)
```




## **Conclusions**

Summarize what you could and could not conclude based on your experiments.
In this section, you can add **text**.



## **References**
You can add here the citations of books, websites, or academic papers, etc.